In [ ]:
import pandas as pd
import string
from fuzzywuzzy import process
import re
import numpy as np

# Function to correct city name
correct_cities = ["NewSouthWales","Victoria","Queensland","NorthTeritory","Tasmania","AustralianCapitalTeritory","WesternAustralia"]  # Add all the cities you want to consider

def correct_city_name(name, correct_cities, min_score=80):
    correct_name = process.extractOne(name, correct_cities)
    if correct_name[1] >= min_score:
        return correct_name[0]
    return name

# Function to get state from postcode
def get_state(postcode):
    postcode_ranges = {
        "ACT": [(2600, 2618), (2900, 2920)],
        "NSW": [(1000, 1999), (2000, 2599), (2620, 2899), (2921, 2999)],
        "NT": [(800, 899), (900, 999)],
        "QLD": [(4000, 4999)],
        "SA": [(5000, 5799), (5800, 5999)],
        "TAS": [(7000, 7799), (7800, 7999)],
        "VIC": [(3000, 3999), (8000, 8999)],
        "WA": [(6000, 6797), (6800, 6999)]
    }
    for state, ranges in postcode_ranges.items():
        if any(start <= int(postcode) <= end for start, end in ranges):
            return state
    return "Unknown state"

# Load your excel file
df = pd.read_excel("C://Users//masst//OneDrive//Desktop//address Cleansing//Sample_data.xlsx")  # replace 'your_file.xlsx' with your actual file name

# Initialize empty lists to hold the postcode and state
postcodes = []
states = []
unit=[]
strt=[]
addr=[]
# For each address in the excel file
for address in df['Address']:  # replace 'Address' with your actual address column name
    # Remove punctuation and split
    #address = address.translate(str.maketrans('', '', string.punctuation))
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    address = address.translate(translator)
    
    address_parts = address.split()
    b1=address_parts.copy()
    u1=address_parts[0]
    u2=address_parts[1]
    contains_digit1= any(character.isdigit() for character in u1)
    contains_digit2= any(character.isdigit() for character in u2)
    if contains_digit1==True and contains_digit2==True:
        unit.append(u1)
        strt.append(u2)
        b1.remove(u1)
        b1.remove(u2)
    elif contains_digit1==True and contains_digit2==False:
        unit.append('')
        strt.append(u1)
        b1.remove(u1)
    else:
        unit.append('Missing Unit number')
        strt.append('Missing Street number')        
    if address_parts[-1].isdigit():
        postcode=address_parts[-1]
        flag=1
    elif address_parts[-2].isdigit():
        postcode=address_parts[-2]
        flag=1
    else:
        flag=0
        postcodes.append('Missing Postcode')
    
    if flag==1:
        state = get_state(postcode)
        if state != "Unknown state":
            postcodes.append(postcode)
            states.append(state)
            b1.remove(postcode)
        else:
            postcodes.append('Invalid Postcode')
            states.append('Invalid Postcode')

    if flag==0:
        d=b1[-1]
        if d in ("ACT","NSW","VIC","NT","TAS","WA","QLD","SA"):
            states.append(d)
            b1.remove(d)
        else:
            s4=''.join(b1)
            d=correct_city_name(s4,correct_cities)
            if d==s4:
                states.append('Missing value for state')
            else:
                d= re.sub(r"(\w)([A-Z])", r"\1 \2", d)
                print(d)
                matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', d)
                s6=matches
                print(s6)
                if d=="New South Wales":
                    states.append("NSW")
                 
                elif d=="Queensland":
                  states.append("QLD")
               
                elif d=="Victoria":
                    states.append("VIC")
                
                elif d=="Australian Capital Teritory":
                    states.append("ACT")
                 
                elif d=="North Teritory":
                    states.append("NT")
                  
                elif d=="Tasmania":
                    states.append("TAS")
              
                elif d=="Western Australia":
                    states.append("WT")
                elif d=="Southern Australia":
                    states.append("SA")
                   
    t5=b1[0] + '  '+ b1[1] + ' & ' +b1[2]
    print(t5)
    addr.append(t5)
    
df['Unit Number']=unit
df['Street Number']=strt
df['Street and Suburb Name']=addr
df['States']=states
df['Postcode'] = postcodes
df.replace('', np.nan, inplace=True)

# Concatenate the columns (except the first one - 'Address') with ',' as a delimiter and ignore empty values:

df['Street and Suburb Name'] = df['Street and Suburb Name'].apply(lambda s: ' '.join(word.title() for word in s.split()))

df['States']=df['States'].str.upper()

df['Combined'] = df.iloc[:, 1:].apply(lambda row: ', '.join(row.dropna().astype(str)), axis=1)
df.loc[df.apply(lambda row: row.astype(str).str.contains('Missing').any(), axis=1), 'Combined'] = 'Invalid Address'


# Save the updated dataframe back to the Excel file
df.to_excel("C://Users//masst//OneDrive//Desktop//address Cleansing//Sample_data.xlsx", index=False)
